In [15]:
# 라이브러리 
import requests
import pandas as pd
import io
import time
import requests
from bs4 import BeautifulSoup
import numpy as np

In [44]:
# pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.


## 네이버 증권 크롤링 함수

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_financial_summary(stock_code):
    # 네이버 증권의 특정 종목 페이지 URL
    url = f'https://finance.naver.com/item/main.nhn?code={stock_code}'
    
    # 웹 페이지 요청
    response = requests.get(url)
    response.encoding = 'cp949'
    
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Financial Summary 테이블 찾기
    table = soup.find('table', {'class': 'tb_type1 tb_num tb_type1_ifrs'})
    
    # 테이블 데이터 파싱
    df = pd.read_html(str(table))[0]
    
    # 데이터프레임 정리
    df.columns = df.columns.droplevel(0)  # 상위 MultiIndex 제거
    df.columns = df.columns.droplevel(1) 
    df.set_index(df.columns[0], inplace=True) # 첫 번째 열을 인덱스로 설정
    
    # 숫자 데이터로 변환할 열 선택 및 변환
    numeric_columns = ['2024.03']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    
    # 매출 성장률 계산
    try:
        sales_growth = ((df.iloc[0, 8] - df.iloc[0, 7]) / df.iloc[0, 7]) * 100
    except KeyError:
        sales_growth = float('NaN')  # 매출액 데이터가 없는 경우 NaN 처리
    except np.core._exceptions.UFuncTypeError:
        sales_growth = float('NaN')  # 매출액 데이터가 없는 경우 NaN 처리
    
    # 매출성장률을 새로운 행으로 추가
    df.loc['매출성장률'] = sales_growth
    
    # 필요한 지표들 선택
    df = df.loc[['영업이익률', '순이익률', '매출액', '부채비율', 'PER(배)', 'PBR(배)', 'ROE(지배주주)', '매출성장률']]
    df=df[['2024.03']]
    
    return df


## 기업별 종목코드 불러오기

In [11]:
# 저장된 종목코드 불러오기
code_data = pd.read_csv('code.csv')
code_data = code_data[['종목코드']]

## 모든 기업별로 재무제표 합치기

In [20]:
# 재무제표 크롤링
for idx, firm_code in enumerate(code_data['종목코드']):
    print(idx + 1, '/', len(code_data['종목코드']))
    try:
        # 종목별 재무제표 크롤링
        try:
            sub_finance_table = get_financial_summary(firm_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            sub_finance_table = get_financial_summary(firm_code)

        # 종목별 재무제표 정리
        for num, col in enumerate(sub_finance_table.columns):
            temp = pd.DataFrame({firm_code : sub_finance_table[col]}).T
            temp.columns = [[col] * len(sub_finance_table), temp.columns]
            if num == 0:
                temp_arranged = temp
            else:
                temp_arranged = pd.merge(temp_arranged, temp, how='outer', left_index=True, right_index=True)

        # 재무제표 통합        
        if idx == 0:
            finance_table = temp_arranged
        else:
            finance_table = pd.concat([finance_table, temp_arranged])
    except ValueError:
        continue
    except KeyError:
        continue

# 재무제표 저장
finance_table.to_csv('finance_table.csv')

1 / 2696
2 / 2696
3 / 2696
4 / 2696
5 / 2696
6 / 2696
7 / 2696
8 / 2696
9 / 2696
10 / 2696
11 / 2696
12 / 2696
13 / 2696
14 / 2696
15 / 2696
16 / 2696
17 / 2696
18 / 2696
19 / 2696
20 / 2696
21 / 2696
22 / 2696
23 / 2696
24 / 2696
25 / 2696
26 / 2696
27 / 2696
28 / 2696
29 / 2696
30 / 2696
31 / 2696
32 / 2696
33 / 2696
34 / 2696
35 / 2696
36 / 2696
37 / 2696
38 / 2696
39 / 2696
40 / 2696
41 / 2696
42 / 2696
43 / 2696
44 / 2696
45 / 2696
46 / 2696
47 / 2696
48 / 2696
49 / 2696
50 / 2696
51 / 2696
52 / 2696
53 / 2696
54 / 2696
55 / 2696
56 / 2696
57 / 2696
58 / 2696
59 / 2696
60 / 2696
61 / 2696
62 / 2696
63 / 2696
64 / 2696
65 / 2696
66 / 2696
67 / 2696
68 / 2696
69 / 2696
70 / 2696
71 / 2696
72 / 2696
73 / 2696
74 / 2696
75 / 2696
76 / 2696
77 / 2696
78 / 2696
79 / 2696
80 / 2696
81 / 2696
82 / 2696
83 / 2696
84 / 2696
85 / 2696
86 / 2696
87 / 2696
88 / 2696
89 / 2696
90 / 2696
91 / 2696
92 / 2696
93 / 2696
94 / 2696
95 / 2696
96 / 2696
97 / 2696
98 / 2696
99 / 2696
100 / 2696
101 / 26

In [147]:
pd.read_csv('finance_table.csv')

,Unnamed: 0,2024.03,2024.03.1,2024.03.2,2024.03.3,2024.03.4,2024.03.5
0,주요재무정보,영업이익률,순이익률,매출액,부채비율,PER(배),PBR(배)
1,095570,7.95,2.16,2448.0,305.26,13.03,0.52
2,006840,8.08,3.56,11934.0,319.33,10.23,0.33
3,027410,9.55,9.71,1185.0,14.85,4.76,0.22
4,282330,1.67,1.2,19538.0,214.53,10.62,1.98
...,...,...,...,...,...,...,...
2616,024060,-0.59,0.41,306.0,17.3,179.8,1.84
2617,010240,9.2,9.77,353.0,25.14,5.56,0.62
2618,189980,8.2,3.72,225.0,112.21,12.28,0.91
2619,037440,3.17,5.63,563.0,169.13,13.1,1.19
